# 1. 라이브러리 임포트

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [3]:
# 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm
from transformers import Trainer, TrainingArguments
import re
import pyarrow as pa
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.model_selection import train_test_split
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')

# 2. 데이터불러오기 및 EDA

In [4]:
data = pd.read_csv('/content/drive/MyDrive/아이펠톤/DKTC data/train.csv')

# 3. 데이터 전처리및 타입변경

In [5]:
data.rename(columns={'class':'labels'}, inplace=True)

In [6]:
# remove '\n' in sentence
import re
def preprocess(sentence):
    return re.sub('\n', '', sentence)

data['conversation'] = data['conversation'].apply(lambda x: preprocess(x))

In [7]:
label_list = {'기타 괴롭힘 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '협박 대화':3}
label_list

{'기타 괴롭힘 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '협박 대화': 3}

In [8]:
data['labels'] = data['labels'].apply(lambda x: label_list[x])

In [9]:
# convert to Huggingface dataset
import pyarrow as pa
from datasets import Dataset

data = Dataset(pa.Table.from_pandas(data))

# **klue/roberta-base**

In [ ]:
model_path = "klue/roberta-base"

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = 4)

In [ ]:
def transform(data):
  return tokenizer(
      data['conversation'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_data = data.map(transform, batched=True)

from sklearn.model_selection import train_test_split
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

{'idx': [0, 1, 2], 'label': [3, 3, 0], 'conversation': ['지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게. 정말 잘못했습니다. 너가 선택해. 너가 죽을래 네 가족을 죽여줄까. 죄송합니다. 정말 잘못했습니다. 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야. 선택 못하겠습니다. 한번만 도와주세요. 그냥 다 죽여버려야겠군. 이의 없지? 제발 도와주세요.', '길동경찰서입니다.9시 40분 마트에 폭발물을 설치할거다.네?똑바로 들어 한번만 더 얘기한다.장난전화 걸지 마시죠. 9시 40분 마트에 폭발물이 터지면 다 죽는거야. 장난전화는 업무방해죄에 해당됩니다.판단은 너에게 달려있다. 길동경찰서에도 폭발물 터지면 꽤나 재미있겠지.선생님 진정하세요.난 이야기했어. 경고했다는 말이야.', '너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.그만해. 니들 놀리는거 재미없어.지영아 너가 키 160이지? 그럼 재는 160도 안돼는거네?너 군대도 안가고 좋겠다.니들이 나 작은데 보태준거 있냐?난쟁이들도 장가가고하던데. 너도 희망을 가져봐 더이상 하지마라. 그 키크는 수술도 있대잖아? 니네 엄마는 그거 안해주디?나람 해줬어. 저 키로 어찌살아.제발 그만 괴롭히라고!']}
{'input_ids': [[0, 3660, 743, 4201, 2138, 11287, 2448, 7245, 20336, 2205, 2259, 575, 2179, 2116, 35, 6525, 18, 7822, 11800, 18, 1565, 2069, 568, 2460, 4638, 1565, 2118, 3616, 2299, 2118, 3863, 2170, 27063, 2318, 1897, 35, 4229, 11287, 10592, 2088, 1335, 2318, 18, 3944, 4389, 2371, 2219, 3606, 18, 743, 2116, 4

In [ ]:
metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

In [ ]:
output_dir = '/content/drive/MyDrive/아이펠톤/미니아펠톤/klue roberta-base'

training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = 16, 
    per_device_eval_batch_size = 16, 
    num_train_epochs = 3,
    weight_decay = 0.01
)

In [ ]:
trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, conversation. If idx, conversation are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3555
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 669
  Number of trainable parameters = 110621188


Epoch,Training Loss,Validation Loss,F1
1,No log,0.259334,0.903797
2,No log,0.233285,0.926582


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, conversation. If idx, conversation are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 395
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, conversation. If idx, conversation are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 395
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/아이펠톤/미니아펠톤/klue roberta-base/checkpoint-500
Configuration saved in /content/drive/MyDrive/아이펠톤/미니아펠톤/klue roberta-base/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/아이펠톤/미니아펠톤/klue roberta-base/checkpoin

Epoch,Training Loss,Validation Loss,F1
1,No log,0.259334,0.903797
2,No log,0.233285,0.926582
3,0.365600,0.308988,0.913924


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, conversation. If idx, conversation are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 395
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=669, training_loss=0.30527725276747447, metrics={'train_runtime': 1049.2003, 'train_samples_per_second': 10.165, 'train_steps_per_second': 0.638, 'total_flos': 2806129794723840.0, 'train_loss': 0.30527725276747447, 'epoch': 3.0})

# **beomi/KcELECTRA-base**

In [ ]:
model_path = "beomi/KcELECTRA-base"

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = 4)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.weight', 'classifi

In [ ]:
def transform(data):
  return tokenizer(
      data['conversation'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_data = data.map(transform, batched=True)

from sklearn.model_selection import train_test_split
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = '/content/drive/MyDrive/아이펠톤/미니아펠톤/beomi/KcELECTRA-base'

training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = 16, 
    per_device_eval_batch_size = 16, 
    num_train_epochs = 3,
    weight_decay = 0.01
)

In [ ]:
trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: idx, conversation. If idx, conversation are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3555
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 669
  Number of trainable parameters = 124548100


Epoch,Training Loss,Validation Loss,F1
1,No log,0.438296,0.881013


The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: idx, conversation. If idx, conversation are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 395
  Batch size = 16


Epoch,Training Loss,Validation Loss,F1
1,No log,0.438296,0.881013
2,No log,0.399888,0.870886
3,0.606800,0.340235,0.903797


The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: idx, conversation. If idx, conversation are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 395
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/아이펠톤/미니아펠톤/beomi/KcELECTRA-base/checkpoint-500
Configuration saved in /content/drive/MyDrive/아이펠톤/미니아펠톤/beomi/KcELECTRA-base/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/아이펠톤/미니아펠톤/beomi/KcELECTRA-base/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: idx, conversation. If idx, conversation are not expected by `ElectraForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num 

TrainOutput(global_step=669, training_loss=0.5045497820159958, metrics={'train_runtime': 1036.1587, 'train_samples_per_second': 10.293, 'train_steps_per_second': 0.646, 'total_flos': 2806129794723840.0, 'train_loss': 0.5045497820159958, 'epoch': 3.0})

# **beomi/KcT5-dev(실패)**

In [ ]:
!pip install flax

In [ ]:
model_path = "beomi/KcT5-dev"

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, num_labels = 4, from_flax=True)

In [ ]:
def transform(data):
  return tokenizer(
      data['conversation'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      max_length = 512
      )

encoded_data = data.map(transform, batched=True)

from sklearn.model_selection import train_test_split
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

In [ ]:
metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = '/content/drive/MyDrive/아이펠톤/미니아펠톤/model_data'

training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = 8, 
    per_device_eval_batch_size = 8, 
    num_train_epochs = 3,
    weight_decay = 0.01
)

In [ ]:
trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)
trainer.train()

# **skt/kobert-base-v1(실패)**

In [ ]:
!pip install sentencepiece

In [ ]:
model_path = "skt/kobert-base-v1"

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = 4)

In [ ]:
def transform(data):
  return tokenizer(
      data['conversation'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      max_length = 512
      )

encoded_data = data.map(transform, batched=True)

from sklearn.model_selection import train_test_split
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

In [ ]:
metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = '/content/drive/MyDrive/아이펠톤/미니아펠톤/beomi/model_data'

training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = 8, 
    per_device_eval_batch_size = 8, 
    num_train_epochs = 3,
    weight_decay = 0.01
)

In [ ]:
trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)
trainer.train()

# **xlm-roberta-large(실패)**

In [ ]:
model_path = "xlm-roberta-large"

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = 4)

In [ ]:
def transform(data):
  return tokenizer(
      data['conversation'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_data = data.map(transform, batched=True)

from sklearn.model_selection import train_test_split
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

In [ ]:
metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

output_dir = '/content/drive/MyDrive/아이펠톤/미니아펠톤/beomi/model_data'

training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = 4, 
    per_device_eval_batch_size = 4, 
    num_train_epochs = 3,
    weight_decay = 0.01
)

In [13]:
trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: idx, conversation. If idx, conversation are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3555
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2667
  Number of trainable parameters = 559894532


Epoch,Training Loss,Validation Loss


RuntimeError: ignored